# Checkpoints

Sometimes, it might be useful to store some checkpoints while executing an algorithm. In particular, if a run is very time-consuming. 
**pymoo** offers to resume a run by serializing the algorithm object and loading it. Resuming runs from checkpoints is possible 

- the functional way by calling the `minimize` method, 
- the object-oriented way by repeatedly calling the `next()` method or 
- from a text file ([Biased Initialization](../customization/initialization.ipynb) from `Population` )

## Functional

In [1]:
import dill
import numpy as np

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_problem
from pymoo.optimize import minimize
from pymoo.termination.max_gen import MaximumGenerationTermination

problem = get_problem("zdt1", n_var=5)

algorithm = NSGA2(pop_size=100)

res = minimize(problem,
               algorithm,
               ('n_gen', 5),
               seed=1,
               copy_algorithm=False,
               verbose=True)

with open("checkpoint", "wb") as f:
    dill.dump(algorithm, f)

with open("checkpoint", 'rb') as f:
    checkpoint = dill.load(f)
    print("Loaded Checkpoint:", checkpoint)

# only necessary if for the checkpoint the termination criterion has been met
checkpoint.termination = MaximumGenerationTermination(20)

res = minimize(problem,
               checkpoint,
               seed=1,
               copy_algorithm=False,
               verbose=True)

n_gen  |  n_eval  | n_nds  |      igd      |       gd      |       hv     
     1 |      100 |      6 |  0.5914067243 |  2.8577180757 |  0.0841819857
     2 |      200 |     12 |  0.4142923277 |  2.5677806357 |  0.1929910317
     3 |      300 |     13 |  0.4141164945 |  1.8140218582 |  0.1929910317
     4 |      400 |     13 |  0.3260510318 |  1.4284805268 |  0.2393608464
     5 |      500 |     12 |  0.2863716201 |  0.9389805558 |  0.2798618546


Loaded Checkpoint: <pymoo.algorithms.moo.nsga2.NSGA2 object at 0x7f05f2ebe490>
     6 |      600 |     16 |  0.2032207467 |  1.1781204920 |  0.3565210005
     7 |      700 |     19 |  0.1769910504 |  1.0917100947 |  0.3846180138
     8 |      800 |     21 |  0.1650278469 |  0.8854634936 |  0.4100198098
     9 |      900 |     22 |  0.1497465622 |  0.4296217574 |  0.4171669870


    10 |     1000 |     25 |  0.1385935347 |  0.3126485838 |  0.4364806508
    11 |     1100 |     28 |  0.1197443704 |  0.1506553570 |  0.4631916155
    12 |     1200 |     32 |  0.0967014276 |  0.1272461455 |  0.5030211647
    13 |     1300 |     26 |  0.0760029935 |  0.0924710716 |  0.5358790734
    14 |     1400 |     33 |  0.0650366349 |  0.0796895797 |  0.5562899448
    15 |     1500 |     40 |  0.0525439361 |  0.0823186520 |  0.5794508388


    16 |     1600 |     48 |  0.0430276743 |  0.0772072430 |  0.5942008130
    17 |     1700 |     45 |  0.0343731737 |  0.0416826975 |  0.6076420316
    18 |     1800 |     51 |  0.0289535770 |  0.0300533296 |  0.6176606634
    19 |     1900 |     60 |  0.0240942124 |  0.0272274925 |  0.6267873202
    20 |     2000 |     69 |  0.0199349997 |  0.0232941258 |  0.6334560769


## Object Oriented

In [2]:
import dill
import numpy as np

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_problem
from pymoo.factory import get_termination
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

problem = get_problem("zdt1", n_var=5)

algorithm = NSGA2(pop_size=100)

algorithm.setup(problem, seed=1, termination=('n_gen', 20))

for k in range(5):
    algorithm.next()
    print(algorithm.n_gen)

    with open("checkpoint", "wb") as f:
        dill.dump(algorithm, f)
    
    
with open("checkpoint", 'rb') as f:
    checkpoint = dill.load(f)
    print("Loaded Checkpoint:", checkpoint)

while checkpoint.has_next():
    checkpoint.next()
    print(checkpoint.n_gen)

2
3


4
5


6


Loaded Checkpoint: <pymoo.algorithms.moo.nsga2.NSGA2 object at 0x7f05f3000be0>
7
8
9
10
11
12
13
14


15
16
17
18
19
20
21


## From a Text File

First, load the data from a file. Usually, this will include the variables `X`, the objective values `F` (and the constraints `G`). Here, they are created randomly. Always make sure the `Problem` you are solving would return the same values for the given `X` values. Otherwise the data might be misleading for the algorithm.

(This is not the case here. It is really JUST for illustration purposes)

In [3]:
import numpy as np
from pymoo.problems.single import G1

problem = G1()

N = 300
np.random.seed(1)
X = np.random.random((N, problem.n_var))

# here F and G is re-evaluated - in practice you want to load them from files too
F, G = problem.evaluate(X, return_values_of=["F", "G"])

Then, create a population object using your data:

In [4]:
from pymoo.core.evaluator import Evaluator
from pymoo.core.population import Population
from pymoo.problems.static import StaticProblem

# now the population object with all its attributes is created (CV, feasible, ...)
pop = Population.new("X", X)
pop = Evaluator().eval(StaticProblem(problem, F=F, G=G), pop)

And finally run it with a non-random initial population `sampling=pop`:

In [5]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize

# the algorithm is now called with the population - biased initialization
algorithm = GA(pop_size=100, sampling=pop)

res = minimize(problem,
               algorithm,
               ('n_gen', 10),
               seed=1,
               verbose=True)

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min     |     f_gap    
     1 |        0 |  0.000000E+00 |  0.1192400898 | -1.037973E+00 | -3.869005E+00 |  1.113099E+01
     2 |      100 |  0.000000E+00 |  0.000000E+00 | -2.236718E+00 | -3.889330E+00 |  1.111067E+01
     3 |      200 |  0.000000E+00 |  0.000000E+00 | -2.803012E+00 | -5.580025E+00 |  9.4199754224
     4 |      300 |  0.000000E+00 |  0.000000E+00 | -3.429410E+00 | -5.816498E+00 |  9.1835016334
     5 |      400 |  0.000000E+00 |  0.000000E+00 | -4.082023E+00 | -6.561619E+00 |  8.4383812986
     6 |      500 |  0.000000E+00 |  0.000000E+00 | -4.843246E+00 | -7.183582E+00 |  7.8164176555
     7 |      600 |  0.000000E+00 |  0.000000E+00 | -5.568205E+00 | -7.316943E+00 |  7.6830565678


     8 |      700 |  0.000000E+00 |  0.000000E+00 | -6.151748E+00 | -7.532310E+00 |  7.4676900556
     9 |      800 |  0.000000E+00 |  0.000000E+00 | -6.651238E+00 | -7.768584E+00 |  7.2314162193
    10 |      900 |  0.000000E+00 |  0.000000E+00 | -7.038396E+00 | -8.441010E+00 |  6.5589896578
